In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
system_name = "Wray Farms"
location = "United Kingdom"
base_url = "https://pvoutput.org/list.jsp?p={}&id=13666&sid=11542&gs=0&v=0&o=date&d=desc"
pages_to_scrape = 5

In [3]:
options = Options()
options.binary_location = r"C:\Program Files\chrome for testing\chrome-win64\chrome.exe"
options.add_argument("--headless=new")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36")
service = Service(executable_path=r"C:\Program Files\chrome for testing\chrome-win64\chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)

In [4]:
all_data = []

for page in range(pages_to_scrape):
    url = base_url.format(page)
    print(f"📄 Scraping page {page + 1} from: {url}")
    
    try:
        driver.get(url)
        time.sleep(6)
        soup = BeautifulSoup(driver.page_source, "html.parser")
        table = soup.find("table", id="tbl_main")

        if not table:
            print(f"⚠️ No table found on page {page + 1}. Retrying once...")
            time.sleep(5)
            driver.get(url)
            soup = BeautifulSoup(driver.page_source, "html.parser")
            table = soup.find("table", id="tbl_main")

        if table:
            rows = table.find_all("tr")[1:]
            for row in rows:
                cols = [td.get_text(strip=True).replace(',', '') for td in row.find_all("td")]
                if len(cols) >= 9:
                    cols.append(system_name)
                    cols.append(location)
                    all_data.append(cols)
        else:
            print(f"❌ Failed again: No table found on page {page + 1}")
    except Exception as e:
        print(f"🚨 Error scraping page {page + 1}: {e}")

driver.quit()

📄 Scraping page 1 from: https://pvoutput.org/list.jsp?p=0&id=13666&sid=11542&gs=0&v=0&o=date&d=desc
📄 Scraping page 2 from: https://pvoutput.org/list.jsp?p=1&id=13666&sid=11542&gs=0&v=0&o=date&d=desc
📄 Scraping page 3 from: https://pvoutput.org/list.jsp?p=2&id=13666&sid=11542&gs=0&v=0&o=date&d=desc
📄 Scraping page 4 from: https://pvoutput.org/list.jsp?p=3&id=13666&sid=11542&gs=0&v=0&o=date&d=desc
📄 Scraping page 5 from: https://pvoutput.org/list.jsp?p=4&id=13666&sid=11542&gs=0&v=0&o=date&d=desc


In [5]:
header = [
    "Date", "Generated", "Efficiency", "Exported", "Peak Power",
    "Peak Time", "Conditions", "Temperature", "Comments",
    "System Name", "Location"
]

In [6]:
df = pd.DataFrame(all_data, columns=header)
df.drop(columns=["Exported", "Temperature", "Comments", "Peak Power", "Peak Time", "Conditions"], inplace=True)
print(df.head())

       Date   Generated   Efficiency System Name        Location
0  30/03/25  141.365kWh  2.827kWh/kW  Wray Farms  United Kingdom
1  29/03/25  106.771kWh  2.135kWh/kW  Wray Farms  United Kingdom
2  28/03/25   91.659kWh  1.833kWh/kW  Wray Farms  United Kingdom
3  27/03/25  104.243kWh  2.085kWh/kW  Wray Farms  United Kingdom
4  26/03/25   89.894kWh  1.798kWh/kW  Wray Farms  United Kingdom


In [25]:
page6 = pd.read_csv("WFSS_page6.csv")
page7 = pd.read_csv("WFSS_page7.csv")
page8 = pd.read_csv("WFSS_page8.csv")
page9 = pd.read_csv("WFSS_page9.csv")
page10 = pd.read_csv("WFSS_page10.csv")

In [27]:
page6["System Name"] = "Wray Farms"
page6["Location"] = "United Kingdom"
page7["System Name"] = "Wray Farms"
page7["Location"] = "United Kingdom"
page8["System Name"] = "Wray Farms"
page8["Location"] = "United Kingdom"
page9["System Name"] = "Wray Farms"
page9["Location"] = "United Kingdom"
page10["System Name"] = "Wray Farms"
page10["Location"] = "United Kingdom"

In [29]:
master_df = pd.concat([df, page6, page7, page8, page9, page10], ignore_index=True)

In [31]:
master_df

,Date,Generated,Efficiency,System Name,Location
0,30/03/25,141.365kWh,2.827kWh/kW,Wray Farms,United Kingdom
1,29/03/25,106.771kWh,2.135kWh/kW,Wray Farms,United Kingdom
2,28/03/25,91.659kWh,1.833kWh/kW,Wray Farms,United Kingdom
3,27/03/25,104.243kWh,2.085kWh/kW,Wray Farms,United Kingdom
4,26/03/25,89.894kWh,1.798kWh/kW,Wray Farms,United Kingdom
...,...,...,...,...,...
395,9/26/2023,123.231,2.46462,Wray Farms,United Kingdom
396,9/25/2023,171.482,3.42964,Wray Farms,United Kingdom
397,9/24/2023,86.931,1.73862,Wray Farms,United Kingdom
398,9/23/2023,140.38,2.8076,Wray Farms,United Kingdom


In [33]:
master_df["System Size (kW)"] = 50

In [35]:
master_df.to_csv("daily_Wray_Farms_Solar_System_master.csv", index=False)